# <font color='#2F4F4F'>AfterWork Data Science: Recurrent Neural Networks with Python - Project</font>

## <font color='#2F4F4F'>Step 1. Business Understanding </font>

### a) Specifying the Research Question

Build a recurrent neural networks model that will be used to predict Tesla stock prices in 2017 using data from 2012-2016

### b) Defining the Metric for Success

Build a neural networks regression model with an RSME  less than 10% of the target mean.

### c) Understanding the Context 

Stockpy is an innovative fintech enabling financial prosperity for the entire population. It is a venture funded startup based in Palo Alto bringing world-class financial experiences to a continually growing customer base. As Stockpy enters an expansion phase for innovative fintech product offerings, it aims to enhance the enormous value in data processing and analysis for continuous growth and success.

As a Finance Data Scientist for Stockpy, you provide leadership to turn cutting-edge technology into actionable insights; unlocking the power of data that provides value to business decisions and customer service enhancements.

### d) Recording the Experimental Design

* Business Understanding
* Data Exploration
* Data Preparation
* Data Modeling and Evaluation
* Summary of Findings and Recommendation
* Challenging the Solution


## <font color='#2F4F4F'>Step 2. Data Importation</font>

In [2]:
# Importing standard libraries
import numpy as np
import pandas as pd

In [26]:
# Importing our dataset
train = pd.read_csv("https://bit.ly/38dSbSb")
test = pd.read_csv("https://bit.ly/3kW8E0K")

## <font color='#2F4F4F'>Step 3. Data Exploration</font>

In [34]:
# Sample dataset
# ---
#
train.sample()

,Date,Open,High,Low,Close,Adj Close,Volume
1175,2016-09-02,40.466,40.639999,39.240002,39.556,39.556,29887000


In [4]:
test.sample()

,Date,Open,High,Low,Close,Adj Close,Volume
100,2020-05-29,161.75,167.0,160.841995,167.0,167.0,59062500


In [35]:
train.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [27]:
from datetime import timedelta
import datetime

# df['Year'] = pd.to_datetime(df['Date'],format='%I:%M:%S %p')

# df['date_Year'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
train['Year'] = pd.DatetimeIndex(train['Date']).year
train['Month'] = pd.DatetimeIndex(train['Date']).month
train['Day'] = pd.DatetimeIndex(train['Date']).day

test['Year'] = pd.DatetimeIndex(test['Date']).year
test['Month'] = pd.DatetimeIndex(test['Date']).month
test['Day'] = pd.DatetimeIndex(test['Date']).day

In [28]:
train=train.drop(columns=['Date'])
test=test.drop(columns=['Date'])

In [10]:
train.sample(5)

,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day
13,5.362000,5.442000,5.320000,5.354000,5.354000,2973000,2012,1,23
793,40.540001,40.667999,39.166000,39.466000,39.466000,39610500,2015,3,2
492,29.516001,30.926001,29.264000,30.492001,30.492001,52475000,2013,12,17
1117,43.900002,45.153999,43.532001,43.574001,43.574001,20965000,2016,6,13
279,7.660000,7.800000,7.610000,7.690000,7.690000,4834000,2013,2,13


In [11]:
train.describe()

,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1.258000e+03,1258.000000,1258.000000,1258.000000
mean,31.997898,32.558671,31.403696,31.989245,31.989245,2.551310e+07,2014.003180,6.543720,15.724165
std,16.911515,17.148543,16.649245,16.903500,16.903500,2.259037e+07,1.413647,3.426707,8.734085
min,5.324000,5.370000,4.528000,4.558000,4.558000,1.824500e+06,2012.000000,1.000000,1.000000
25%,8.403000,8.556000,8.197000,8.379500,8.379500,1.059400e+07,2013.000000,4.000000,8.000000
50%,39.411000,40.123998,38.709000,39.427000,39.427000,2.007650e+07,2014.000000,7.000000,16.000000
75%,45.214499,45.934999,44.303000,45.076502,45.076502,3.245150e+07,2015.000000,10.000000,23.000000
max,57.534000,58.284000,56.080002,57.208000,57.208000,1.858195e+08,2016.000000,12.000000,31.000000


In [29]:
train=train.drop(columns=['Adj Close'])
test=test.drop(columns=['Adj Close'])

In [5]:
# Statistical summary
# ---
#
train.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [7]:
test.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [13]:
train.describe()

,Open,High,Low,Close,Volume,Year,Month,Day
count,1258.000000,1258.000000,1258.000000,1258.000000,1.258000e+03,1258.000000,1258.000000,1258.000000
mean,31.997898,32.558671,31.403696,31.989245,2.551310e+07,2014.003180,6.543720,15.724165
std,16.911515,17.148543,16.649245,16.903500,2.259037e+07,1.413647,3.426707,8.734085
min,5.324000,5.370000,4.528000,4.558000,1.824500e+06,2012.000000,1.000000,1.000000
25%,8.403000,8.556000,8.197000,8.379500,1.059400e+07,2013.000000,4.000000,8.000000
50%,39.411000,40.123998,38.709000,39.427000,2.007650e+07,2014.000000,7.000000,16.000000
75%,45.214499,45.934999,44.303000,45.076502,3.245150e+07,2015.000000,10.000000,23.000000
max,57.534000,58.284000,56.080002,57.208000,1.858195e+08,2016.000000,12.000000,31.000000


## <font color='#2F4F4F'>Step 4. Data Preparation</font>

In [23]:
# Getting our train dataset
# ---
#
# X_train = np.array(train.drop(columns = ['Close']))
# y_train = np.array(train['Close'])

# X_test = np.array(test.drop(columns = ['Close']))
# y_test = np.array(test['Close'])

In [30]:
# Performing Feature scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# fitting and transforming X_train while transforming X_test
train_set_scaled = scaler.fit_transform(train)
test_set_scaled = scaler.transform(test)

In [31]:
# Creating a dataset with 60 timesteps and 1 output
# ---
#
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(train_set_scaled[i-60:i, 0])
    y_train.append(train_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [32]:
# Reshaping 
# ---
#
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

##  <font color='#2F4F4F'>Step 5. Data Modeling</font>

In [33]:


# Building the RNN
# ---
#
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [34]:
# Initialising the RNN
# ---
#
regressor = Sequential()

# Adding 4 LSTM layers and some Dropout regularisation
# ---
#
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = False))
regressor.add(Dropout(0.2))

# Adding the output layer
# ---
regressor.add(Dense(units = 1))

In [35]:
# Compile the RNN 
# ---
#
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [36]:
# Fitting the RNN to the training set
# ---
#
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
38/38 [==============================] - 12s 114ms/step - loss: 0.1910
Epoch 2/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0492
Epoch 3/100
38/38 [==============================] - 5s 125ms/step - loss: 0.0383
Epoch 4/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0367
Epoch 5/100
38/38 [==============================] - 4s 115ms/step - loss: 0.0341
Epoch 6/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0317
Epoch 7/100
38/38 [==============================] - 4s 112ms/step - loss: 0.0318
Epoch 8/100
38/38 [==============================] - 5s 134ms/step - loss: 0.0290
Epoch 9/100
38/38 [==============================] - 4s 112ms/step - loss: 0.0309
Epoch 10/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0266
Epoch 11/100
38/38 [==============================] - 4s 113ms/step - loss: 0.0277
Epoch 12/100
38/38 [==============================] - 4s 115ms/step - loss: 0.0257
Epoch 13/100

In [41]:
# Making predictions: Getting the predicted stock price for 2017
# ---
#
X_test = []
y_test = []
for i in range(60, 212):
    X_test.append(test_set_scaled[i-60:i, 0])
    y_test.append(test_set_scaled[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)

predicted_price = regressor.predict(X_test)

In [42]:
# Visualizing the results
# ---
#
print(predicted_price)

[[2.7175136]
 [3.353227 ]
 [2.5592377]
 [3.5025744]
 [2.678086 ]
 [3.6459584]
 [2.9354146]
 [3.6412256]
 [3.423969 ]
 [4.0765505]
 [3.6334953]
 [4.0102987]
 [4.124481 ]
 [3.4200623]
 [4.236677 ]
 [3.3174787]
 [4.242687 ]
 [3.3540778]
 [4.2594166]
 [3.928872 ]
 [4.1192827]
 [4.2970886]
 [3.2219017]
 [4.2292976]
 [4.0554476]
 [3.8114488]
 [4.259817 ]
 [3.8893666]
 [4.20082  ]
 [4.1617956]
 [4.053427 ]
 [4.066928 ]
 [4.170642 ]
 [4.154809 ]
 [4.0635834]
 [4.240613 ]
 [4.0126305]
 [4.264736 ]
 [4.0948   ]
 [4.1706247]
 [4.1368675]
 [4.1223116]
 [4.3074436]
 [4.2033033]
 [4.2836847]
 [4.2684584]
 [4.227601 ]
 [4.3910837]
 [4.269083 ]
 [4.4712486]
 [4.305238 ]
 [4.453826 ]
 [4.167199 ]
 [4.5576944]
 [4.1603813]
 [4.55519  ]
 [4.3112144]
 [4.4911757]
 [4.3853264]
 [4.449361 ]
 [4.321291 ]
 [4.5062714]
 [4.2529488]
 [4.54045  ]
 [4.442255 ]
 [4.6020393]
 [4.5576506]
 [4.646406 ]
 [4.619087 ]
 [4.6509686]
 [4.6445436]
 [4.6920414]
 [4.669588 ]
 [4.6892643]
 [4.6733847]
 [4.6909065]
 [4.684411 ]

In [ ]:
# Model Evaluation
# ---
#
YOUR CODE GOES HERE

## <font color='#2F4F4F'>Step 6. Summary of Findings and Recommendation</font>

Your observation goes here...

## <font color='#2F4F4F'>Step 7. Challenging our Solution</font>

#### a) Did we have the right question?


#### b) Did we have the right data?


#### c) What can be done to improve the solution?
